
# INSTALLATION

In [1]:
!pip install aif360
!pip install fairlearn

In [2]:
!apt-get install -jre
!java -version

E: Command line option 'j' [from -jre] is not understood in combination with the other options.
openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [3]:
!pip install h2o

In [4]:
!pip install xlsxwriter

In [5]:
!pip install BlackBoxAuditing

#IMPORTS

In [6]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import BlackBoxAuditing
import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
import aif360
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover, Reweighing, LFR,OptimPreproc
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import sys
sys.path.append("../")
import os



/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [7]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp19sl5gpa
  JVM stdout: /tmp/tmp19sl5gpa/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp19sl5gpa/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.5
H2O_cluster_version_age:,13 days
H2O_cluster_name:,H2O_from_python_unknownUser_5a57tk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


#**************************LOADING DATASET*******************************

In [8]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [9]:
# tr=pd.read_csv(r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test/Test50.csv')
# tr
# tester= BinaryLabelDataset(favorable_label=1,
#                                 unfavorable_label=0,
#                                 df=tr,
#                                 label_names=['two_year_recid'],
#                                 protected_attribute_names=['race'],
#                                 unprivileged_protected_attributes=[[0]],
#                                 privileged_protected_attributes=[[1]])
# tester
# DIR = DisparateImpactRemover(repair_level=1.0)
# DI_Test = DIR.fit_transform(tester)


# GBM REWEIGHING



In [12]:
for i in range(11,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)

  #********************************************************binary labels for Reweighing*************************************************************
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]
  bldTrain= BinaryLabelDataset(favorable_label=1,
                                unfavorable_label=0,
                                df=train,
                                label_names=['two_year_recid'],
                                protected_attribute_names=['race'],
                                unprivileged_protected_attributes=[[0]],
                                privileged_protected_attributes=[[1]])
  
  bldTest=  BinaryLabelDataset(favorable_label=1,
                                unfavorable_label=0,
                                df=test,
                                label_names=['two_year_recid'],
                                protected_attribute_names=['race'],
                                unprivileged_protected_attributes=[[0]],
                                privileged_protected_attributes=[[1]])
  #*******************************************************DI Remover instance**************************************************************
  Reweigh = Reweighing(unprivileged_groups=advantagedGroup ,
                privileged_groups=disadvantagedGroup )

  RW_Train = Reweigh .fit_transform(bldTrain )
  RW_Test = Reweigh .fit_transform(bldTest)

  #getting weights from reweighing to be added to Train set
  weight= pd.DataFrame(data=RW_Train.instance_weights, columns= ['weight'])
  weight=h2o.H2OFrame(weight)
  
  #*****************************************Repaired Train and Test Set*******************************************************
  train=  pd.DataFrame(np.hstack([RW_Train .labels,RW_Train .features]),columns=train.columns)
  test=  pd.DataFrame(np.hstack([RW_Test .labels,RW_Test .features]),columns= test.columns)
  # TotalRepairedDF= pd.concat([RepairedTrain ,RepairedTest ])


  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************


  #Transforming the Train and Test Set to BinaryLabel

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  ## ********************Checking Bias Repaired Data********************************
  DataBias_Checker = BinaryLabelDatasetMetric(BLD_Test    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)

  dsp= DataBias_Checker .statistical_parity_difference() 
  dif= DataBias_Checker.consistency()  
  ddi= DataBias_Checker.disparate_impact() 

  print('The Statistical Parity diference is = {diff}'.format(diff=  dsp  ))
  print('Individual Fairness is = {IF}'.format( IF=  dif ))
  print('Disparate Impact is = {IF}'.format( IF=   ddi  ))
  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "two_year_recid"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  #adding weights to train set
  Train= Train.cbind(weight)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()
  aml = H2OAutoML(max_models=10, nfolds=10, include_algos=['GBM'] , stopping_metric='AUTO') #verbosity='info',,'GBM', 'DRF'
  aml.train(x=x, y=y, training_frame=Train,weights_column ='weight')
  best_model= aml.leader
  # a.model_performance()
  #**********************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS****************************
  #predicted labels
  gbm_Predictions= best_model.predict(Test)
  gbm_Predictions= gbm_Predictions.as_data_frame()
  predicted_df= test.copy()
  predicted_df['two_year_recid']= gbm_Predictions.predict.to_numpy()

  # ********************COMPUTE DISCRIMINATION*****************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])

  # # Workbook= pd.ExcelFile(r'/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/BaseLines/GBM/gbm_Results.xlsx')
  # OldDF= excelBook.get_sheet_by_name("Violent")#pd.read_excel(Workbook,sheet_name='Violent')
  #load workbook
  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Reweighing/GBM/gbm_Results.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF,
                                          'DATA_SP':dsp,'DATA_CONS':dif,'DATA_DI':ddi})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Reweighing/GBM/gbm_Results.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Parse progress: |█████████████████████████████████████████████████████████| 100%
The Statistical Parity diference is = 0.055323802725587914
Individual Fairness is = [0.81147132]
Disparate Impact is = 1.4779361846571624
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.7817955112219451
Parse progress: |█████████████████████████████████████████████████████████| 100%
The Statistical Parity diference is = 0.08433684794672587
Individual Fairness is = [0.80099751]
Disparate Impact is = 1.6951400800457406
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |███████████████████████████████

#LOGISTIC REGRESSION REWEIGHING

In [11]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path)
  first_column = train.pop('two_year_recid')
  train.insert(0, 'two_year_recid', first_column)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Violent/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path)
  first_column = test.pop('two_year_recid')
  test.insert(0, 'two_year_recid', first_column)

   #********************************************************binary labels for DI Remover*************************************************************

  bldTrain= BinaryLabelDataset(favorable_label=1,
                                unfavorable_label=0,
                                df=train,
                                label_names=['two_year_recid'],
                                protected_attribute_names=['race'],
                                unprivileged_protected_attributes=[[0]],
                                privileged_protected_attributes=[[1]])
  
  bldTest=  BinaryLabelDataset(favorable_label=1,
                                unfavorable_label=0,
                                df=test,
                                label_names=['two_year_recid'],
                                protected_attribute_names=['race'],
                                unprivileged_protected_attributes=[[0]],
                                privileged_protected_attributes=[[1]])
  #*******************************************************Reweighing  instance**************************************************************
  Reweigh = Reweighing(unprivileged_groups=advantagedGroup ,
                privileged_groups=disadvantagedGroup )

  RW_Train = Reweigh .fit_transform(bldTrain )
  RW_Test = Reweigh .fit_transform(bldTest)

  #getting weights from reweighing to be added to Train set
  weight= pd.DataFrame(data=RW_Train.instance_weights, columns= ['weight'])
  weight=h2o.H2OFrame(weight)
  
  #*****************************************Repaired Train and Test Set*******************************************************
  train=  pd.DataFrame(np.hstack([RW_Train .labels,RW_Train .features]),columns=train.columns)
  test=  pd.DataFrame(np.hstack([RW_Test .labels,RW_Test .features]),columns= test.columns)
  # TotalRepairedDF= pd.concat([RepairedTrain ,RepairedTest ])

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************


  #Transforming the Train and Test Set to BinaryLabel
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  # class Train(StandardDataset):
  #     def __init__(self,label_name= 'two_year_recid',
  #                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


  #         super(Train, self).__init__(df=train  , label_name=label_name ,
  #             favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
  #             privileged_classes=privileged_classes ,
  #            )





  # BLD_Train= Train(protected_attribute_names= ['race'],
  #                        privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  ## ********************Checking Bias in Data********************************
  DataBias_Checker = BinaryLabelDatasetMetric(BLD_Test    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)

  dsp= DataBias_Checker .statistical_parity_difference() 
  dif= DataBias_Checker.consistency()  
  ddi= DataBias_Checker.disparate_impact() 

  print('The Statistical Parity diference is = {diff}'.format(diff=  dsp  ))
  print('Individual Fairness is = {IF}'.format( IF=  dif ))
  print('Disparate Impact is = {IF}'.format( IF=   ddi  ))
  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "two_year_recid"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  #adding weights to train set
  Train= Train.cbind(weight)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()
  LogReg = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0)
  LogReg.train(x=x, y=y, training_frame=Train,weights_column ='weight')

  LogReg_Predictions= LogReg.predict(Test)
  LogReg_Predictions= LogReg_Predictions.as_data_frame()
  # *************************REPLACE LABELS OF DUPLICATED TEST SET WITH PREDICTIONS**************************************
  predicted_df= test.copy()
  predicted_df['two_year_recid']= LogReg_Predictions.predict.to_numpy()

  # ***************************COMPUTE DISCRIMINATION********************************

  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class PredTest(StandardDataset):
      def __init__(self,label_name= 'two_year_recid',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PredTest, self).__init__(df=predicted_df  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredTest= PredTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])


  excelBook= load_workbook(r'/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Reweighing/LogReg/LR_Results.xlsx')
  Violent= excelBook['Violent']
  data= Violent.values

  # Get columns
  columns = next(data)[0:]
 
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF,
                                          'DATA_SP':dsp,'DATA_CONS':dif,'DATA_DI':ddi})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/Reweighing/LogReg/LR_Results.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Violent', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Parse progress: |█████████████████████████████████████████████████████████| 100%
The Statistical Parity diference is = 0.07377721761620619
Individual Fairness is = [0.79975062]
Disparate Impact is = 1.5999252695633608
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Possession Child Pornography, c_charge_desc_Depriv LEO of Protect/Communic, c_charge_desc_Poss Of RX Without RX, c_charge_desc_Offn Against Intellectual Prop, c_charge_desc_DWLS Canceled Disqul 1st Off, c_charge_desc_Theft/To Deprive, c_charge_desc_Battery On Parking Enfor Speci, c_charge_desc_Poss Drugs W/O A Prescription, c_charge_desc_Solic to Commit Battery, c_charge_desc_Lewd/Lasciv Molest Elder Persn, c_charge_desc_Refuse Submit Blood/Breath Test, c_charge_desc_Tamper With Victim, c_charge_desc_Possess Weapon On School Prop, c_charge_desc_Purchase/P/W/Int Cannabis, c_charge_desc_Intoxicated/Safety Of Another, c_charge_desc_Sale/Del Counterfeit Cont Subs, c_charge_desc_Use of Anti-Shoplifting Device, c_charge_desc_Misuse Of 911 Or E911 System, c_charge_desc_Obtain Control Substance By Fraud, c_charge_desc_Felony DUI - Enhanced, c_charge_desc_Carjacking with a Firearm, c_charge_desc_Soliciting For Prostitution, c_charge_desc_Murder 

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.7805486284289277
Parse progress: |█████████████████████████████████████████████████████████| 100%
The Statistical Parity diference is = 0.0816729917773561
Individual Fairness is = [0.81496259]
Disparate Impact is = 1.7573313782991202
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Attempt Armed Burglary Dwell, c_charge_desc_Posses/Disply Susp/Revk/Frd DL, c_charge_desc_Use Scanning Device to Defraud, c_charge_desc_Violation Of Boater Safety Id, c_charge_desc_Robbery W/Deadly Weapon, c_charge_desc_Aggravated Battery (Firearm/Actual Possession), c_charge_desc_Hiring with Intent to Defraud, c_charge_desc_Structuring Transactions, c_charge_desc_DUI - Property Damage/Personal Injury, c_charge_desc_Principal In The First Degree, c_charge_desc_Poss Meth/Diox/Meth/Amp (MDMA), c_charge_desc_Poss Unlaw Issue Id, c_charge_desc_Possession Of Anabolic Steroid, c_charge_desc_Att Burgl Struc/Conv Dwel/Occp, c_charge_desc_Trespass Structure/Conveyance, c_charge_desc_Possession Of Diazepam, c_charge_desc_Possession Of Phentermine, c_charge_desc_Burglary Structure Occupied, c_charge_desc_Cash Item w/Intent to Defraud, c_charge_desc_Burglary Structure Assault/Batt, c_charge_desc_Deliver Cocaine 1000FT Store, c_charge_desc_Harm Publ

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.7793017456359103
Parse progress: |█████████████████████████████████████████████████████████| 100%
The Statistical Parity diference is = 0.07829013092170986
Individual Fairness is = [0.82618454]
Disparate Impact is = 1.7535425101214575
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


...

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.7855361596009975
Parse progress: |█████████████████████████████████████████████████████████| 100%
The Statistical Parity diference is = 0.04737999391089612
Individual Fairness is = [0.81072319]
Disparate Impact is = 1.3777238403451997
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Use Scanning Device to Defraud, c_charge_desc_Bribery Athletic Contests, c_charge_desc_Robbery W/Deadly Weapon, c_charge_desc_Structuring Transactions, c_charge_desc_Fail Sex Offend Report Bylaw, c_charge_desc_Tresspass in Structure or Conveyance, c_charge_desc_D.U.I. Serious Bodily Injury, c_charge_desc_Violation License Restrictions, c_charge_desc_Extradition/Defendants, c_charge_desc_Poss Of RX Without RX, c_charge_desc_Possession Of Fentanyl, c_charge_desc_DUI - Property Damage/Personal Injury, c_charge_desc_False 911 Call, c_charge_desc_Offn Against Intellectual Prop, c_charge_desc_DWLS Canceled Disqul 1st Off, c_charge_desc_Obstruct Fire Equipment, c_charge_desc_Solic to Commit Battery, c_charge_desc_Att Tamper w/Physical Evidence, c_charge_desc_License Suspended Revoked, c_charge_desc_Misuse Of 911 Or E911 System, c_charge_desc_Threat Public Servant, c_charge_desc_Possess w/I/Utter Forged Bills, c_charge_desc_Lease For Purpose Tr

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.800498753117207
Parse progress: |█████████████████████████████████████████████████████████| 100%
The Statistical Parity diference is = 0.06211562531714082
Individual Fairness is = [0.81620948]
Disparate Impact is = 1.524328954882924
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


Dropping bad and constant columns: [c_charge_desc_Alcoholic Beverage Violation-FL, c_charge_desc_Hiring with Intent to Defraud, c_charge_desc_DUI/Property Damage/Persnl Inj, c_charge_desc_Offer Agree Secure/Lewd Act, c_charge_desc_Unauth C/P/S Sounds>1000/Audio, c_charge_desc_Possession Of Anabolic Steroid, c_charge_desc_Tampering with a Victim, c_charge_desc_False Info LEO During Invest, c_charge_desc_Computer Pornography, c_charge_desc_Poss Drugs W/O A Prescription, c_charge_desc_Carjacking w/o Deadly Weapon, c_charge_desc_Possess Weapon On School Prop, c_charge_desc_Purchase/P/W/Int Cannabis, c_charge_desc_Intoxicated/Safety Of Another, c_charge_desc_Manage Busn W/O City Occup Lic, c_charge_desc_Sale/Del Counterfeit Cont Subs, c_charge_desc_Possession Of Phentermine, c_charge_desc_Agg Abuse Elderlly/Disabled Adult, c_charge_desc_Unl/Disturb Education/Instui, c_charge_desc_Deliver Cocaine 1000FT Store, c_charge_desc_Harm Public Servant Or Family, c_charge_desc_Del Cannabis For Consid

glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.7880299251870324
